In [80]:
import kagglehub
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

# Download latest version
path = kagglehub.dataset_download("atharvaingle/crop-recommendation-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/galich/.cache/kagglehub/datasets/atharvaingle/crop-recommendation-dataset/versions/1


In [44]:
# Load the dataset
df = pd.read_csv(f"{path}/Crop_recommendation.csv")

# Display the first few rows
print(df.head())

# Check column names
print(df.columns)


    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice
Index(['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label'], dtype='object')


In [45]:
# Encode labels as integers
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])  # Converts crop names to integers

# Standardize input features
scaler = StandardScaler()
x = scaler.fit_transform(df.drop(columns=['label']))

# One-hot encode labels for multi-class classification
encoder = OneHotEncoder(sparse_output=False)
y_one_hot = encoder.fit_transform(df[['label']])  # Convert labels to one-hot

# Convert to float
x = x.astype(float)
y_one_hot = y_one_hot.astype(float)

print(x.shape, y_one_hot.shape)

(2200, 7) (2200, 22)


In [65]:
def outer_product(vec_a, vec_b):
    assert len(vec_a) > 0 and len(vec_b) > 0, "Input vectors must not be empty"

    out = np.zeros((len(vec_a), len(vec_b)))

    for i in range(len(vec_a)):  # Iterate over vec_a (rows)
        for j in range(len(vec_b)):  # Iterate over vec_b (columns)
            out[i][j] = vec_a[i] * vec_b[j]  # Multiply element-wise

    return out


def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Stabilize softmax
    return exp_x / np.sum(exp_x)  # Convert scores into probabilities

def neural_network(input, weights):
    return softmax(np.dot(input, weights))  # Matrix multiplication + softmax


In [78]:
# Assuming num_inputs = 7 (features) and num_classes = 22 (crops)
num_inputs = 7
num_classes = 22

weights = np.random.randn(num_inputs, num_classes) * 0.01  # Small random weights
alpha = 0.01  # Learning rate
epochs = 100

for epoch in range(epochs):
    total_loss = 0

    for i in range(len(x)):  # Loop over dataset
        input = x[i]  # Features
        true = y_one_hot[i]  # One-hot encoded label

        # Forward pass
        pred = neural_network(input, weights)

        # Compute loss (cross-entropy)
        loss = (pred - true) ** 2  # Avoid log(0)
        total_loss += loss

        # Compute gradient (backpropagation)
        delta = pred - true  # Gradient of softmax + cross-entropy
        weight_deltas = outer_product(input, delta) # Outer product
        # Update weights
        weights -= alpha * weight_deltas

    print(f"Epoch {epoch+1}, Loss: {total_loss}")

# Save weights to JSON
weights_dict = {"weights": weights.tolist()}  # Convert numpy array to list for JSON
with open("crop-predict-app/weights.json", "w") as f:
    json.dump(weights_dict, f)

print("Final Weights saved to weights.json")


Epoch 1, Loss: [22.16867461 68.46823189 81.50197219 38.70491256 58.68368714 68.16430546
 66.8745343  31.64610583 80.94951724 48.06247786 78.91870226 85.68411172
 63.96821262 80.06570799 77.15162642 55.62734828 68.98767887 68.11525695
 72.62634169 76.83348494 51.06164239 71.46468094]
Epoch 2, Loss: [17.80332445 36.19441638 64.37510738 16.22172459 29.35531063 37.50567351
 41.82415759 23.59576058 66.2721067  28.81560758 55.52097719 71.2493308
 36.18813395 67.1967669  56.05522432 32.09616025 47.36668691 45.28282876
 46.13513856 56.16363921 36.71581608 55.38200792]
Epoch 3, Loss: [16.38340214 20.47084103 54.91287469  9.88506897 21.44247853 24.62114095
 32.88579264 20.50238707 59.70681242 24.53490852 43.82180746 58.81901645
 27.79106912 61.15821834 42.76296531 26.86578203 41.63749482 38.09633411
 35.20766689 45.82112416 34.10426347 46.71854205]
Epoch 4, Loss: [15.15487726 13.43044546 48.5055631   6.73859356 17.64961167 18.12571866
 27.69797852 18.51899084 55.98429773 21.50261151 37.14930435 

In [76]:
def predict_crop(input_features):
    input_features = scaler.transform([input_features])  # Normalize input
    probs = neural_network(input_features[0], weights)  # Get probability distribution
    probs = softmax(probs)  # Apply softmax to ensure probabilities sum to 1

    crop_names = le.inverse_transform(np.arange(len(probs)))  # Get all crop names
    predictions = [{"crop": crop, "probability": prob} for crop, prob in zip(crop_names, probs)]

    return sorted(predictions, key=lambda x: x["probability"], reverse=True)  # Sort by highest probability

# Example prediction
test_input = [150, 48, 17, 23, 73, 7, 66]  # Example soil & climate values
predicted_crops = predict_crop(test_input)

print(predicted_crops)

[{'crop': 'cotton', 'probability': 0.11206894162041381}, {'crop': 'maize', 'probability': 0.043177079164667384}, {'crop': 'watermelon', 'probability': 0.042286942811309604}, {'crop': 'coffee', 'probability': 0.04225488760861917}, {'crop': 'jute', 'probability': 0.042238523486823165}, {'crop': 'banana', 'probability': 0.04223766631498966}, {'crop': 'muskmelon', 'probability': 0.04223388302435986}, {'crop': 'blackgram', 'probability': 0.04223349979367939}, {'crop': 'rice', 'probability': 0.04223347212793774}, {'crop': 'chickpea', 'probability': 0.04223347081776787}, {'crop': 'orange', 'probability': 0.042233469951693636}, {'crop': 'kidneybeans', 'probability': 0.04223346958159296}, {'crop': 'mothbeans', 'probability': 0.04223346942617914}, {'crop': 'grapes', 'probability': 0.04223346940204834}, {'crop': 'lentil', 'probability': 0.042233469386743744}, {'crop': 'mungbean', 'probability': 0.042233469385530055}, {'crop': 'papaya', 'probability': 0.04223346936603279}, {'crop': 'apple', 'proba

/home/galich/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
